# Building a Simple LLM Application with LCEL

In [16]:
!pip install Ollama

!pip install colab-xterm #this will install colab-xterm
%load_ext colabxterm

The colabxterm extension is already loaded. To reload it, use:
  %reload_ext colabxterm


In [20]:
%xterm

In [20]:
!pip install langchain langchain-community langchain-core

In [22]:
import os
from dotenv import load_dotenv

In [24]:
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [25]:
pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 3.1 MB/s eta 0:00:00


In [26]:
#from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

model=ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x79ec31d8aef0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x79ec31d8a230>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [32]:
# Runnables
from langchain_core.messages import HumanMessage, SystemMessage
messages =[
    SystemMessage(content="Translate the following from English to Hindi."), # What the model shoudl do
    HumanMessage(content="Hello, How are you?") # what the question we ask
]


result=model.invoke(messages)

In [33]:
result

AIMessage(content='नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nLet me break it down:\n\n* **नमस्ते** (Namaste) - Hello\n* **आप** (aap) - You (formal)\n* **कैसे** (kaise) - How\n* **हैं** (hain) - Are \n\n\n \n', response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 23, 'total_tokens': 105, 'completion_time': 0.149090909, 'prompt_time': 0.00013715, 'queue_time': 0.014228409, 'total_time': 0.149228059}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-670591cd-15fb-4d7c-9030-e59bbfb53875-0', usage_metadata={'input_tokens': 23, 'output_tokens': 82, 'total_tokens': 105})

In [34]:
#Output Parsers - to get only the output we wnat
from langchain_core.output_parsers import StrOutputParser
parser= StrOutputParser()
parser.invoke(result)

'नमस्ते, आप कैसे हैं? (Namaste, aap kaise hain?) \n\n\nLet me break it down:\n\n* **नमस्ते** (Namaste) - Hello\n* **आप** (aap) - You (formal)\n* **कैसे** (kaise) - How\n* **हैं** (hain) - Are \n\n\n \n'

# LCEL

Using LCEL we can chain the components

In [35]:
chain=model|parser
chain.invoke(messages)

'नमस्ते, आप कैसे हैं? \n'

In [37]:
## Prompt Templates

from langchain_core.prompts import ChatPromptTemplate
generic_template = "Translate the following into {language}:"
prompt=ChatPromptTemplate.from_messages(
    [("system", generic_template),("user", "{text}")]
)

In [40]:
result2=prompt.invoke({"language":"Hindi", "text":"Hello, How are you?"})

In [41]:
result2.to_messages()

[SystemMessage(content='Translate the following into Hindi:'),
 HumanMessage(content='Hello, How are you?')]

In [42]:
chain=prompt|model|parser
chain.invoke({"language":"Hindi", "text":"Hello, How are you?"})

"नमस्ते, आप कैसे हैं ? \n\n(Namaste, aap kaise hain?) \n\n\nLet me know if you'd like me to translate anything else! 😊 \n"

# Langserve - Deploy as REST API

In [44]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.1 MB/s eta 0:00:00


In [45]:
!pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.7 MB/s eta 0:00:00


In [53]:
!pip install langserve
!pip install sse_starlette

In [54]:
!python app.py

INFO:     Started server process [24473]
INFO:     Waiting for application startup.

 __          ___      .__   __.   _______      _______. _______ .______     ____    ____  _______
|  |        /   \     |  \ |  |  /  _____|    /       ||   ____||   _  \    \   \  /   / |   ____|
|  |       /  ^  \    |   \|  | |  |  __     |   (----`|  |__   |  |_)  |    \   \/   /  |  |__
|  |      /  /_\  \   |  . `  | |  | |_ |     \   \    |   __|  |      /      \      /   |   __|
|  `----./  _____  \  |  |\   | |  |__| | .----)   |   |  |____ |  |\  \----.  \    /    |  |____
|_______/__/     \__\ |__| \__|  \______| |_______/    |_______|| _| `._____|   \__/     |_______|

LANGSERVE: Playground for chain "/chain/" is live at:
LANGSERVE:  │
LANGSERVE:  └──> /chain/playground/
LANGSERVE:
LANGSERVE: See all available routes at /docs/

LANGSERVE: ⚠️ Using pydantic 2.8.2. OpenAPI docs for invoke, batch, stream, stream_log endpoints will not be generated. API endpoints and playground should work as e

In [56]:
 !npm install localtunnel
 !python app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com


up to date, audited 23 packages in 554ms

3 packages are looking for funding
  run `npm fund` for details

2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
34.80.77.110
your url is: https://chilly-bats-jam.loca.lt


In [59]:
!pip install streamlit

In [ ]:
!streamlit run app1.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.80.77.110
your url is: https://short-moles-exist.loca.lt
